<h1>Databases I</h1>

<p>A database is a systematic collection of information that supports manipulation and retrieval. Databases are important in programming because they allow for storage of information that can be retrived whenever the program is running. For our Virtual Bank project, so far our bank loses all data once it stops execution (when the user chooses to exit). With databases we can persist the records we create such that when we run it again, we still have the accounts and transactions from the previous session.</p>

<h2>SQLite database</h2>

<p>Python comes with a built-in database called SQLite, which can be used to quickly set up a persistence layer (means of persisting data) in your program. It is a file-based database that does not require a separate application or server to use.</p>

In [2]:
# get the relevant module
import sqlite3
# create a connection object; this creates a file
dbConn = sqlite3.connect('data/kingsley.db')

<p>You can also supply the special name :memory: to create a database in RAM.</p>

In [3]:
# a connection object specifying a RAM-based DB; this will not persist after closing the program
ramConn = sqlite3.connect(':memory:')

<p>A cursor object is created from the connection object. This is what is used to execute queries, fetch results and get information on the database. The connection object is used to save changes and close the connection.</p>

In [4]:
# create the cursors
dbCursor = dbConn.cursor()
ramCursor = ramConn.cursor()

<h3>Queries</h3>

<p>Queries are commands that are run against a database to manipulate the data stored in it. Common queries are <b>CREATE</b>, <b>INSERT</b>, <b>SELECT</b>, <b>UPDATE</b> and <b>DELETE</b>.</p>

<p>Each value stored in an SQLite databasehas one of the following storage classes:
    <ul>
        <li><b>NULL</b> The value is a NULL value.</li>
        <li><b>INTEGER</b> The value is a signed integer, stored in 1, 2, 3, 4, 6, or 8 bytes depending on the magnitude of the value.</li>
        <li><b>REAL</b> The value is a floating point value, stored as an 8-byte IEEE floating point number.</li>
        <li><b>TEXT</b> The value is a text string, stored using the database encoding (UTF-8, UTF-16BE or UTF-16LE).</li>
        <li><b>BLOB</b> The value is a blob of data, stored exactly as it was input.</li>
    </ul>
</p>

In [5]:
# define the CREATE query for a table - accounts
queryStr = 'CREATE TABLE accounts (name text, ss_id text, phone text, balance integer, age integer, password text)'
# execute the query using each cursor
try:
    dbCursor.execute(queryStr)
except sqlite3.OperationalError:
    print('The table already exists')
ramCursor.execute(queryStr)

In [6]:
# save (commit) the changes using the connection object
dbConn.commit()
ramConn.commit()

In [7]:
# create a record
data = ('Annika Bolund', '197702038877', '0729776303', 50000, 30, 'bolund')
queryStr = 'INSERT INTO accounts VALUES (?, ?, ?, ?, ?, ?)'
dbCursor.execute(queryStr, data)
ramCursor.execute(queryStr, data)

In [8]:
# check affected rows
print(f'Affected rows in file DB: {dbCursor.rowcount}')
print(f'Affected rows in RAM DB: {ramCursor.rowcount}')

Affected rows in file DB: 1
Affected rows in RAM DB: 1


In [9]:
# add multiple records
data = [('Mary Svensson', '197709230011', '0733402387', 45000, 32, 'svensson'), ('Kate Smith', '198811180922', '0753001928', 109000, 27, 'smith'), 
    ('Jackson Wafula', '190210179112', '0721109833', 80000, 59, 'jackula')]
dbCursor.executemany(queryStr, data)
ramCursor.executemany(queryStr, data)

In [10]:
# check affected rows
print(f'Affected rows in file DB: {dbCursor.rowcount}')
print(f'Affected rows in RAM DB: {ramCursor.rowcount}')

Affected rows in file DB: 3
Affected rows in RAM DB: 3


In [11]:
# save (commit) the changes using the connection object
dbConn.commit()
ramConn.commit()

In [12]:
# read records from the database
queryStr = 'SELECT * FROM accounts'
dbCursor.execute(queryStr)
ramCursor.execute(queryStr)

<p>For SQLite specifically, rowcount does not return the number of rows for SELECT queries. We can therefore just use the built-in len() to find out the number of rows fetched.</p>

In [13]:
# fetch the results
rows = dbCursor.fetchall()
rowsRam = ramCursor.fetchall()
# check if we have any rows
if len(rows) > 0:
    print(rows)
else:
    print('No records found')
    
if len(rowsRam) > 0:
    print(rowsRam)
else:
    print('No records found in RAM')

[('Annika Bolund', '197702038877', '0729776303', 50000, 30, 'bolund'), ('Mary Svensson', '197709230011', '0733402387', 45000, 32, 'svensson'), ('Kate Smith', '198811180922', '0753001928', 109000, 27, 'smith'), ('Jackson Wafula', '190210179112', '0721109833', 80000, 59, 'jackula')]
[('Annika Bolund', '197702038877', '0729776303', 50000, 30, 'bolund'), ('Mary Svensson', '197709230011', '0733402387', 45000, 32, 'svensson'), ('Kate Smith', '198811180922', '0753001928', 109000, 27, 'smith'), ('Jackson Wafula', '190210179112', '0721109833', 80000, 59, 'jackula')]


<p>Test the persistence by closing the program and then running it again. Do not insert any data, just read and see if each database retains the data.</p>

In [14]:
# fetch one specific result
queryStr = 'SELECT * FROM accounts WHERE name = ?'
dbCursor.execute(queryStr, ('Annika Bolund',))
row = dbCursor.fetchone()
print(row)

('Annika Bolund', '197702038877', '0729776303', 50000, 30, 'bolund')


In [15]:
# fetch a batch based on a condition
queryStr = 'SELECT * FROM accounts WHERE balance > ?'
dbCursor.execute(queryStr, (75000,))
rows = dbCursor.fetchall()
if len(rows) > 0:
    print(rows)
else:
    print('No records found')

[('Kate Smith', '198811180922', '0753001928', 109000, 27, 'smith'), ('Jackson Wafula', '190210179112', '0721109833', 80000, 59, 'jackula')]


In [16]:
# update a result
queryStr = 'UPDATE accounts SET balance = ? WHERE name = ?'
dbCursor.execute(queryStr, (65000, 'Annika Bolund'))
# save (commit) the changes using the connection object
dbConn.commit()

In [17]:
# check to confirm changes
queryStr = 'SELECT * FROM accounts WHERE name = ?'
dbCursor.execute(queryStr, ('Annika Bolund',))
row = dbCursor.fetchone()
print(row)

('Annika Bolund', '197702038877', '0729776303', 65000, 30, 'bolund')


In [18]:
# delete a record
queryStr = 'DELETE FROM accounts WHERE name = ?'
dbCursor.execute(queryStr, ('Mary Svensson',))
# save (commit) the changes using the connection object
dbConn.commit()

In [19]:
# check to confirm changes
queryStr = 'SELECT * FROM accounts WHERE name = ?'
dbCursor.execute(queryStr, ('Mary Svensson',))
row = dbCursor.fetchone()
print(row)

None
